In [20]:
import numpy as np
import pylab 
import copy
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.aqua.components.optimizers import SLSQP
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.core import Hamiltonian, QubitMappingType
from qiskit.aqua.algorithms import VQE, NumPyEigensolver


In [21]:
molecule = 'H .0 .0 -{0}; Li .0 .0 {0}'
distances = np.arange(0.5, 4.25, 0.25) # ^-10
vqe_energies = []
hf_energies = []
exact_energies = []

In [22]:
for i,d in enumerate(distances):
    print('step', i)
    
    # set up the experiment
    driver = PySCFDriver(molecule.format(d/2),basis='sto3g')
    qmolecule = driver.run()
    operator = Hamiltonian(qubit_mapping=QubitMappingType.PARITY,
                          two_qubit_reduction=True, freeze_core=True,
                          orbital_recutction=[-3,-2])
    qubit_op, aux_ops = operator.run(qmolecule)

    #exact classical result
    exact_result = NumPyMinimumEigensolver(qubit_op, aux_operators=aux_ops).run()
    exacet_result = operator.process_algorithm_result(exact_result)

    #VQE
    optimizer = SLSQP(maxiter=1000)
    initial_state = HartreeFock(operator.melecule_info['num_oribtals'],
                               operator.molecule_infp['num_particle'],
                               qubit_mapping=operator._qubit_mapping,
                               two_qubit_reduction=operator._two_qubit_reduction)
    var_form = UCCSD(num_orbitals=operator.molecule_info['num_orbitals'],
                    num_particles=operator.molecule_info['num_particles'],
                    initial_state=initial_state,
                    qubit_mapping=operator._qubit_mapping,
                    two_qubit_reduction=operator._two_qubit_reduction)
    algo = VQE(qubit_op, var_form, optimizer, aux_operators=aux_ops)
    vqe_result = algo.run(QuantumInstance(BasicAer.get_backend('statevector_simulator')))
    vqe_result = operator.process_algorithm_result(vqe_result)

    exact_energies.append(exact_result.energy)
    vqe_energies.append(vqe_result.energy)
    hf_energies.append(vqe_reuslt.hartree_fock_energy)

step 0


NameError: name 'param' is not defined